# Nomic

## Installation

In [1]:
%pip install -q langchain langchain-community nomic pyPDF2 pandas sentence-transformers transformers torch > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [2]:
import nomic
import time
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders.pdf import PyPDFDirectoryLoader  # Importing PDF loader from Langchain
from langchain.vectorstores import AtlasDB
from langchain.document_loaders import TextLoader
from nomic import atlas
nomic.login('<[place nomi key here]>') #api key to a limited demo account. Make your own account at atlas.nomic.ai

In [3]:
embedd = HuggingFaceEmbeddings()

/tmp/ipykernel_90404/2766102236.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedd = HuggingFaceEmbeddings()
/tmp/ipykernel_90404/2766102236.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedd = HuggingFaceEmbeddings()
/home/vscode/my-envs/03-rag-graph-databases/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console

Note: Create the Nomic example dataset

In [4]:
# Example from Nomic 25k news articles
from nomic import atlas
import pandas

news_articles = pandas.read_csv('https://raw.githubusercontent.com/nomic-ai/maps/main/data/ag_news_25k.csv')

print(news_articles)

dataset = atlas.map_data(data=news_articles, indexed_field='text')
print(dataset)

2024-09-11 03:21:06.009 | WARNING  | nomic.atlas:map_data:128 - An ID field was not specified in your data so one was generated for you in insertion order.


          id                                               text  label
0          0  Nasdaq planning \$100m-share sale The owner of...      2
1          1  Report urges shuttle mission to service Hubble...      3
2          2  Update 5: Euro Reaches New High Against US Dol...      0
3          3  Franz wins downhill, Guay 13th VAL D #39;ISERE...      1
4          4  Creative plans US\$100m advertising campaign f...      3
...      ...                                                ...    ...
24995  24995  F1: British GP 'saved by deal' Formula One bos...      0
24996  24996  Asia Shares Higher on BOJ Survey; Gold Up (Reu...      2
24997  24997  News In Brief The speedy GeForce Go 6800 mobil...      3
24998  24998  US Stocks Gain on Optimism About Election; Nex...      2
24999  24999  Parents to Read 9/11 Victims' Names NEW YORK -...      0

[25000 rows x 3 columns]


2024-09-11 03:21:07.792 | INFO     | nomic.dataset:_create_project:857 - Creating dataset `inquisitive-newton`
2024-09-11 03:21:08.022 | INFO     | nomic.atlas:map_data:140 - Uploading data to Atlas.
100%|██████████| 5/5 [00:01<00:00,  2.83it/s]
2024-09-11 03:21:09.875 | INFO     | nomic.dataset:_add_data:1668 - Upload succeeded.
2024-09-11 03:21:09.878 | INFO     | nomic.atlas:map_data:158 - `botchagalupe/inquisitive-newton`: Data upload succeeded to dataset`
2024-09-11 03:21:11.196 | INFO     | nomic.dataset:create_index:1262 - Created map `inquisitive-newton` in dataset `botchagalupe/inquisitive-newton`: https://atlas.nomic.ai/data/botchagalupe/inquisitive-newton


inquisitive-newton: https://atlas.nomic.ai/data/botchagalupe/inquisitive-newton


Note: Create the Nomic Void dataset

In [5]:
from pypdf import PdfReader

reader = PdfReader("../data/papers/aiayn.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

#print(word_wrap(pdf_texts[0]))
print(pdf_texts[0])


FileNotFoundError: [Errno 2] No such file or directory: '../data/papers/aiayn.pdf'

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

In [ ]:
character_splitter = RecursiveCharacterTextSplitter(
    #separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0,
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

#print(character_split_texts)
for i in range(5):
    print("Chunk#=" + str(i) + " Chunk=" + character_split_texts[i],"\n")

print(f"\nTotal chunks: {len(character_split_texts)}")

Chunk#=0 Chunk=JohnWillis-AI-Frankendraftv21
THERISEOFAI
UnmaskingtheMagicofChatGPT
JohnWillis

JohnWillis-AI-Frankendraftv22
Contents
Contents2Foreword/Testimonials3Prologue4—PartI:HowAIBegan—51|PrometheanFireorPandora’sBox?62|Myth&Legend193|TheLittleEnginesThatCouldn’t264|SecretIntelligence315|Aberdeen376|ProveYou’reNotaRobot447|TheMagicMouseofBellLabs498|TheSummerItStarted559|ThinkinginHuman6110|ArtiﬁcialExpert7211|TheBigBlueBrute81—PartII:HowAILearns—8812|ThePitts8913|TheFrog’sEye9914|Chess&Checkers10715|DeepBluevs.DeepLearning118—PartIII:HowAIReads—12416|TheRayKurzweilShow12517|TeachingAItheABCsand123s131—PartIV:HowAISees—13718|AI’sEyeballs13819|AIsandAVs14420|TheLittleGPUThatCould15321|Fei-FeiLiTeachesAItoSee157—PartV:HowAISpeaks—16822|IfItQuacksLikeaDuck…16923|TheStoryofSiri178Acknowledgments179

JohnWillis-AI-Frankendraftv23
Foreword/Testimonials
DarrenHaasJohnAllsparDr.Woods 

Chunk#=1 Chunk=JohnWillis-AI-Frankendraftv24
Prologue
[storyoffeedingDomenicoLepore’sbookTheHumanCond

In [ ]:
embeddings = embedd.embed_documents(character_split_texts)

In [ ]:
import pandas as pd


df = pd.DataFrame(embeddings)


In [ ]:
from nomic import atlas

dataset = atlas.map_data(data=df

, indexed_field='text')
print(dataset)

2024-09-04 14:48:25.378 | WARNING  | nomic.atlas:map_data:128 - An ID field was not specified in your data so one was generated for you in insertion order.
2024-09-04 14:48:29.307 | INFO     | nomic.dataset:_create_project:857 - Creating dataset `tenacious-rubin`
2024-09-04 14:48:29.894 | INFO     | nomic.atlas:map_data:140 - Uploading data to Atlas.
/home/vscode/.local/lib/python3.12/site-packages/nomic/dataset.py:1485: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  data = pa.Table.from_pandas(data)
1it [00:05,  5.72s/it]
2024-09-04 14:48:36.049 | INFO     | nomic.dataset:_add_data:1668 - Upload succeeded.
2024-09-04 14:48:36.050 | INFO     | nomic.atlas:map_data:158 - `botchagalupe/tenacious-rubin`: Data upload succeeded to dataset`


Exception: Indexing on text not allowed. Valid options are: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149',